<a href="https://colab.research.google.com/github/ipeirotis-org/academiabot/blob/master/05-Time_Series/A-Introduction_to_Time_Series_using_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Time Series and Forecasting

*Based on the book [Introduction to Time Series and Forecasting](https://link.springer.com/book/10.1007/978-3-319-29854-2) by Brockwell and Davis.*

## Why Time Series Analysis Matters

Your manager walks into your office and says:

> *"Sales are up 15% this month compared to last month. Great job!"*

But wait—is that actually good news? What if your product is ice cream, and you're comparing July to June? Or what if you're comparing December retail sales to November? Without understanding **seasonality**, you can't tell whether 15% growth is exceptional, expected, or actually disappointing.

Time series analysis gives you the tools to answer questions like:
- Is this month's performance actually above or below expectations?
- What's the underlying trend, separate from seasonal fluctuations?
- What should we expect next quarter? Next year?
- Are there anomalies that require investigation?

These skills are essential for anyone working with business data—whether you're forecasting demand, planning capacity, managing inventory, or evaluating performance.

## Learning Objectives

By the end of this notebook, you will be able to:

1. **Recognize and plot time-indexed data** using Pandas datetime functionality
2. **Pull external time series data** via APIs and libraries (stocks, FRED economic data)
3. **Normalize time series** by indexing and through inflation adjustment
4. **Diagnose key patterns**: autocorrelation, seasonality, and trend
5. **Apply window functions** (rolling, expanding, exponential) for trend extraction
6. **Decompose time series** into trend, seasonal, and residual components
7. **Change temporal granularity** using resampling

In [ ]:
# You need to change the project_id to your own Google Cloud project
project_id = "nyu-datasets" # <<<<<< CHANGE THIS

In [ ]:
#@title Setup

# This code snippet authenticates the user to access
# Google Cloud services from within Colab. This is
# necessary to interact with services like BigQuery.
!pip install -q google-cloud-bigquery

from google.colab import auth
from google.cloud import bigquery

auth.authenticate_user()

client = bigquery.Client(project=project_id)

!pip install -U -q yfinance fredapi

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

from fredapi import Fred
import yfinance as yf

In [ ]:
#@title Plotting Setup

%config InlineBackend.figure_format = 'retina'

# Change the graph defaults
plt.rcParams['figure.figsize'] = (8, 3)  # Default figure size of 8x3 inches
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.color'] = 'lightgray'
plt.rcParams['font.size'] = 10  # Default font size of 10 points
plt.rcParams['lines.linewidth'] = 1  # Default line width of 1 points
plt.rcParams['lines.markersize'] = 3  # Default marker size of 3 points
plt.rcParams['legend.fontsize'] = 10  # Default legend font size of 10 points

---
## Part 1: What is a Time Series?

A time series is a set of observations $x_t$, each one being recorded at a specific time $t$. In our sessions, we focus on **discrete** time series, where observations are recorded at fixed time intervals (e.g., once an hour, or every 30 seconds, or every 7 days).

We only consider **regular** time series, where the time between observations is constant (i.e., we do not consider account deposits or withdrawals from an ATM that happen at various times; these are examples of an irregular time series).

### The Core Components of a Time Series

Most business time series can be broken down into three components:

| Component | Description | Business Example |
|-----------|-------------|------------------|
| **Trend** | Long-term direction (up, down, flat) | Company revenue growing 8% annually |
| **Seasonality** | Predictable, repeating patterns | Retail sales spike in Q4 every year |
| **Residual** | Random noise after removing trend & seasonality | Unexpected supply chain disruption |

Understanding these components helps us answer questions like:
- "Is our underlying business growing?" (trend)
- "When should we staff up for peak demand?" (seasonality)  
- "Was last Tuesday's drop an anomaly?" (residual)

---
## Part 2: Examples of Time Series

### Example 1: Australian Red Wine Sales (Jan 1980 - Oct 1991)

The file [`australian-wine-sales.txt`](https://storage.googleapis.com/datasets_nyu/australian-wine-sales.txt) contains the monthly sales of Australian red wines. Let's load and explore this classic time series dataset.

In [ ]:
url = "https://storage.googleapis.com/datasets_nyu/australian-wine-sales.txt"
df = pd.read_csv(url, sep='\t')

# The `read_csv` command can read directly from a URL.
# Since the file uses tab characters to separate columns,
# we pass the `sep='\t'` option.

In [ ]:
df.head(10)

In [ ]:
df.plot(
    x = 'Date',
    y = 'Sales'
)

**Interpreting the Australian Wine Sales Plot:**

This plot visually represents monthly Australian red wine sales over time. We can observe two key features:

1. **Upward Trend:** There's a general increase in sales over the years, suggesting growing demand or population growth. This long-term movement is the *trend* component. For a business, identifying this trend is crucial for long-term planning and capacity forecasting.

2. **Seasonal Pattern:** Notice the recurring peaks and troughs within each year. Sales tend to be higher in certain months and lower in others. This predictable, repeating pattern is the *seasonal* component. Understanding this seasonality is vital for inventory management, staffing, and marketing campaigns. The peaks appear to be around July (winter in Australia), and the troughs around January (summer).

### Example 2: NYC Vehicle Accidents

Let's look at a different type of time series—one from BigQuery that you may recognize from earlier work.

In [ ]:
# Get the number of accidents per hour
sql = '''
  SELECT DATE_TRUNC(DATE_TIME, HOUR) AS acc_date, COUNT(*) AS accidents
  FROM collisions.collisions
  GROUP BY acc_date
  ORDER BY acc_date
'''

# Read the results into Pandas
acc_hourly = client.query(sql).to_dataframe()

acc_hourly.plot(
    kind='line',
    x='acc_date',
    y='accidents'
)

**Interpreting the NYC Hourly Accidents Plot:**

This plot shows vehicular accidents in NYC recorded on an hourly basis—a much more granular view than the wine sales.

- **Volatility:** Unlike the relatively smooth wine sales trend, this plot is quite volatile, showing significant fluctuations from hour to hour.
- **Potential Patterns:** While it's hard to see clear long-term trends at this resolution, we can observe periods of higher and lower activity. By aggregating this data, we can reveal more distinct patterns.

### Changing Granularity with Resampling

We can change the granularity of the time series using the `resample()` method. This is one of the most useful time series operations in Pandas.

**Common Frequency Codes for Resampling:**

| Code | Meaning | Example Use |
|------|---------|-------------|
| `'h'` or `'H'` | Hourly | High-frequency trading data |
| `'D'` | Daily (calendar day) | Daily sales reports |
| `'B'` | Business day | Workday-only analysis |
| `'W'` | Weekly (Sunday end) | Weekly summaries |
| `'MS'` | Month start | Monthly reporting (1st of month) |
| `'ME'` | Month end | Monthly reporting (end of month) |
| `'QE'` | Quarter end | Quarterly earnings |
| `'YE'` | Year end | Annual reports |

In [ ]:
# Aggregate to weekly data
acc_weekly = acc_hourly.copy()

# When we want to perform resampling, we need the datetime to be the "index"
acc_weekly = acc_weekly.set_index('acc_date')

# Resample to weekly frequency and sum the accidents
acc_weekly = acc_weekly.resample('W').sum()

# Plot the aggregated data; by default, x-axis is the index
acc_weekly.plot(
    kind='line',
    y='accidents',
    title='NYC Accidents (Weekly)'
)

**Interpreting the Aggregated Plot:**

By aggregating hourly data to weekly, the plot becomes smoother and reveals clearer patterns:
- **Smoothed Trend:** Weekly aggregation helps smooth out hourly noise
- **Reduced Volatility:** Makes it easier to identify underlying movements
- **Tradeoff:** Weekly seasonality (more accidents on certain days) is now hidden

This highlights the importance of choosing the right aggregation level for the patterns you want to study.

### Example 3: US Population

In [ ]:
# We use the thousands=',' option to properly convert population numbers to integers
population = pd.read_csv("https://storage.googleapis.com/datasets_nyu/us-population.txt", sep=' ', thousands=',')
population["Year"] = pd.to_numeric(population["Year"])
population["US_Population"] = pd.to_numeric(population["US_Population"])

population.plot(
    kind = 'line',
    x = 'Year',
    y = 'US_Population',
    marker = 'o'
)

**Interpreting the US Population Plot:**

- **Clear, Smooth Trend:** This is a classic example of a time series dominated by a strong, consistent upward trend. Population growth is generally a very smooth process.
- **Absence of Seasonality:** As expected, annual population data does not show seasonality.
- **Business Relevance:** Businesses operating in the US market would use this data for long-term market size forecasting and strategic planning.

---
## Part 3: Working with Financial Data

### Stock Prices with Yahoo Finance

The `yfinance` library provides easy access to stock price data. If you're interested in financial data analysis, [this article provides a comprehensive guide](https://towardsdatascience.com/a-comprehensive-guide-to-downloading-stock-prices-in-python-2cd93ff821d4).

In [ ]:
stock_df = yf.download(
    tickers = ['GOOG','AAPL','MSFT', 'IBM'],
    interval = '1d',        # download daily prices
    start='2005-01-01',     # fetch prices after 2005
    auto_adjust = True,     # adjust for splits etc
    progress = False        # do not show a progress bar
)['Close']  # Keep only the closing price

display(stock_df)

In [ ]:
stock_df.plot(title='Stock Prices (Raw)')

In [ ]:
stock_df.plot(title='Stock Prices (Raw, Log-scale)', logy=True)

### Normalization: Indexing to a Base Date

Raw stock prices are hard to compare because they started at different levels. A common technique is to **normalize** by dividing all values by the price on a base date. This shows relative performance.

**Why normalize?** If Stock A goes from \$10 to \$20 and Stock B goes from \$100 to \$150, which performed better? Stock A doubled (100% return) while Stock B gained 50%. Normalization makes this clear.

In [ ]:
# Get the first valid value for each stock
first_values = stock_df.bfill().iloc[0]

# Normalize: divide each column by its first value
normalized_stock_df = stock_df / first_values

normalized_stock_df.plot(title='Normalized Stock Prices (Base = First Trading Day)', logy=True)

Now we can clearly see that Apple and Google have significantly outperformed IBM and Microsoft over this period (though Microsoft has caught up recently).

### Calculating Returns

For financial analysis, we often care about **returns** (percentage changes) rather than raw prices. The `pct_change()` method calculates the percentage change from the previous period.

In [ ]:
# Calculate daily returns
daily_returns = stock_df.pct_change()

# Show summary statistics
daily_returns.describe()

In [ ]:
# Visualize the distribution of returns
daily_returns.hist(bins=100, figsize=(10, 6), alpha=0.7)
plt.suptitle('Distribution of Daily Returns')
plt.tight_layout()

---
## Part 4: Economic Data from FRED

The [FRED Economic Data by the Federal Reserve Bank of St Louis](https://fred.stlouisfed.org/) publishes a rich set of 822,000 US and international time series from 110 sources. This is an invaluable resource for business analysis.

In [ ]:
fred = Fred(api_key='c041995ed8b9ab9c3f475e2ca8f7c288')

In [ ]:
# Consumer Price Index for All Urban Consumers, not seasonally-adjusted
# https://fred.stlouisfed.org/series/CPIAUCNS

cpi = fred.get_series('CPIAUCNS')
cpi.plot(title='Consumer Price Index (CPI)')

**Interpreting the CPI Plot:**

- **Upward Trend:** There is a clear and persistent upward trend, indicating that the general price level has increased significantly over time. This is inflation.
- **Varying Slope:** The slope isn't constant—there are periods of higher inflation (steeper slope) and lower inflation (flatter slope).
- **Business Relevance:** Tracking CPI is essential for pricing strategies, cost management, wage negotiations, and understanding purchasing power.

In [ ]:
# Price increases compound exponentially,
# so a log-scale helps us understand relative changes better
cpi.plot(logy=True, title='CPI (Log Scale)')

**Why use a log scale?** On a log scale, a straight line indicates a constant *percentage* growth rate. Deviations from a straight line show periods where inflation was accelerating or decelerating.

In [ ]:
# CPI data is monthly. Calculate the 12-month percentage change
# This is the standard way inflation is reported (year-over-year)
cpi.pct_change(12).plot(title='Annual Inflation Rate (12-month % change in CPI)')

---
## Activity 1: Normalizing with Inflation Adjustment

**Scenario:** You want to analyze whether retail sales have truly grown in "real" terms (after accounting for inflation) or just in nominal terms.

**Tasks:**
1. Retrieve and plot the [Advance Retail Sales: Retail Trade and Food Services](https://fred.stlouisfed.org/series/RSAFSNA) series from FRED. The series code is `'RSAFSNA'`.
2. Normalize the retail sales values to account for inflation using the CPI time series.

**Hint:** To adjust for inflation, divide nominal values by the CPI index. You may need to align the time series (both are monthly, but ensure they match up).

In [ ]:
# Your code here


### Activity 1 Solution

In [ ]:
#@title Activity 1 Solution (click to expand)

# Step 1: Get retail sales data
retail_sales = fred.get_series('RSAFSNA')

# Plot nominal retail sales
retail_sales.plot(title='Retail Sales (Nominal)', label='Nominal')
plt.ylabel('Millions of Dollars')
plt.show()

# Step 2: Normalize for inflation
# First, align the series by using a common date range
# CPI is indexed with a base period (typically 1982-84 = 100)

# Get the CPI value at the start of retail sales data to use as base
common_start = max(retail_sales.index.min(), cpi.index.min())
common_end = min(retail_sales.index.max(), cpi.index.max())

# Filter both series to common range
retail_aligned = retail_sales[common_start:common_end]
cpi_aligned = cpi[common_start:common_end]

# Normalize CPI to first period = 100 for easier interpretation
cpi_normalized = cpi_aligned / cpi_aligned.iloc[0] * 100

# Calculate real (inflation-adjusted) retail sales
real_retail_sales = retail_aligned / cpi_normalized * 100

# Plot both nominal and real
plt.figure(figsize=(10, 4))
retail_aligned.plot(label='Nominal Sales')
real_retail_sales.plot(label='Real Sales (Inflation-Adjusted)')
plt.title('Retail Sales: Nominal vs. Real')
plt.ylabel('Index')
plt.legend()
plt.show()

print("\nKey Insight: Real growth is smaller than nominal growth because")
print("part of the nominal increase is due to inflation, not actual volume growth.")

---
## Part 5: Analyzing Autocorrelation

A commonly analyzed property of a time series is **autocorrelation**—the correlation of the series with lagged versions of itself. Simply stated, high autocorrelation means that if we know the value at time $t$, we can predict well the value at $t+1$.

In [ ]:
# Load the wine sales data and prepare it for analysis
url = "https://storage.googleapis.com/datasets_nyu/australian-wine-sales.txt"
df = pd.read_csv(url, sep='\t')

# Convert the string date to proper datetime format
df["Date"] = pd.to_datetime(df["Date"])
df = df.set_index("Date")
df["Sales"] = pd.to_numeric(df["Sales"])

In [ ]:
# Autocorrelation at lag 1 (correlation between t and t+1)
print(f"Autocorrelation at lag 1: {df['Sales'].autocorr(lag=1):.3f}")
print(f"Autocorrelation at lag 12: {df['Sales'].autocorr(lag=12):.3f}")

**Interpreting Autocorrelation:**
- Lag 1 (~0.73): Sales this month are strongly correlated with sales last month
- Lag 12 (~0.94): Sales this month are *very* strongly correlated with sales from the same month last year—this indicates strong seasonality!

### Lag Plots

A **lag plot** visualizes the relationship between a time series and its lagged values. If there is autocorrelation, we'll see a pattern; if there's no autocorrelation (pure noise), we'll see a random scatter.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

# Lag 1 plot
pd.plotting.lag_plot(df["Sales"], lag=1, c='b', ax=axes[0])
axes[0].set_title(f'Lag 1 (autocorr = {df["Sales"].autocorr(lag=1):.2f})')

# Lag 12 plot
pd.plotting.lag_plot(df["Sales"], lag=12, c='r', ax=axes[1])
axes[1].set_title(f'Lag 12 (autocorr = {df["Sales"].autocorr(lag=12):.2f})')

plt.tight_layout()

Notice that lag 12 shows a tighter correlation (less spread) than lag 1, confirming the strong annual seasonality in wine sales.

#### Interactive Lag Plot
And here is a more interactive plot that allows you to change the lag value using a slider

In [ ]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

lag = widgets.IntSlider(min=1,max=36,value=12)

def update_plot(l):
    fig, ax = plt.subplots(figsize=(5, 5)) # Create a new figure and axes
    # Create a DataFrame for regplot with lagged data
    lagged_data = pd.DataFrame({
        'y(t)': df['Sales'],
        f'y(t + {l})': df['Sales'].shift(-l)
    }).dropna() # Drop rows with NaN values created by shifting

    if not lagged_data.empty:
        sns.regplot(x='y(t)', y=f'y(t + {l})', data=lagged_data, scatter_kws={'color': 'b'}, line_kws={'color': 'red'}, ax=ax)
        ax.set_title(f'Lag Plot with Regression Line (Lag = {l})')
        ax.set_xlabel('y(t)')
        ax.set_ylabel(f'y(t + {l})')
        ax.set_xlim(500, 3000) # Set x-axis limits
        ax.set_ylim(500, 3000) # Set y-axis limits
        ax.grid(True)
        plt.show()
    else:
        print(f"No data available for lag = {l}")
        plt.close(fig) # Close the empty figure

widgets.interact(update_plot, l=lag)

### Autocorrelation Plot (ACF)

The **autocorrelation plot** shows the correlation value for many different lag values at once. This is a powerful diagnostic tool for identifying seasonality.

In [ ]:
pd.plotting.autocorrelation_plot(df["Sales"])

**Interpreting the ACF Plot:**

- **High Correlation at Small Lags:** Sales in one month are strongly correlated with recent months—this suggests a smooth, trending series.
- **Peaks at Lag 12, 24, 36:** The prominent peaks at multiples of 12 confirm strong annual seasonality. Sales are highly correlated with the same month in previous years.
- **Business Insight:** This tells us that past sales (especially 12 months ago) are very good predictors of current sales—crucial for seasonal forecasting and inventory management.

---
## Part 6: Seasonal Decomposition

Now let's formally decompose the time series into its components: **Trend**, **Seasonality**, and **Residual**.

There are two common models:
- **Additive:** $Y_t = T_t + S_t + R_t$ (seasonal effect is constant in absolute terms)
- **Multiplicative:** $Y_t = T_t \times S_t \times R_t$ (seasonal effect scales with the trend)

For the wine sales data, since the seasonal swings get larger as the trend increases, a multiplicative model is more appropriate.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# Decompose with 12-month periodicity (monthly data with annual seasonality)
decomposition = seasonal_decompose(
    x = df['Sales'],
    model='multiplicative',
    period=12,
    extrapolate_trend=24
)

In [ ]:
fig = plt.figure(figsize=(10, 8))
fig = decomposition.plot(
    observed=False,
    seasonal=True,
    trend=True,
    resid=True,
)

**Interpreting the Decomposition:**

- **Trend:** The steady upward movement in wine sales over the period. This is what we'd use for long-term planning.
- **Seasonal:** The repeating annual pattern—higher in winter months (remember, Australia!), lower in summer. This is essential for short-term planning like inventory and staffing.
- **Residual:** What's left after removing trend and seasonality. Ideally, this should look like random noise. Large spikes might indicate anomalies worth investigating.

In [ ]:
# Access individual components
decomposition.trend.plot(title='Trend Component')

In [ ]:
# The seasonal component repeats every 12 months
decomposition.seasonal.head(24).plot(title='Seasonal Component (2 years)')

---
## Part 7: Window Functions for Trend Analysis

Another approach to extracting trends is using **window functions** that operate over a set of continuous time series points. These are especially useful when you want more control over how the trend is calculated.

| Function | Description | Use Case |
|----------|-------------|----------|
| `rolling(n)` | Fixed window of n periods | Smooth out noise, extract trend |
| `expanding()` | Window grows from start | Cumulative statistics |
| `ewm(halflife=n)` | Exponentially weighted | More weight on recent data |

In [ ]:
# Compare different window functions
plt.figure(figsize=(10, 4))

# Raw data (semi-transparent)
df['Sales'].plot(label='Raw', linestyle="--", alpha=0.25)

# 12-month moving average (removes seasonality)
df['Sales'].rolling(12).mean().plot(label='12M Rolling Mean', alpha=0.75)

# Expanding mean (cumulative average from start)
df['Sales'].expanding().mean().plot(label='Expanding Mean', alpha=0.75)

# Exponentially weighted moving average
df['Sales'].ewm(halflife=12).mean().plot(label='EWMA (halflife 12M)', alpha=0.75)

plt.legend(bbox_to_anchor=(1, 0.5))
plt.title('Comparing Window Functions')
plt.tight_layout()

**Key Insight:** The 12-month rolling mean removes the annual seasonality entirely (since it averages over a complete cycle), leaving just the underlying trend. This is why the rolling window size should match the seasonal period.

---
## Activity 2: Analyze NYC Accidents

**Scenario:** The NYC Department of Transportation wants to understand accident patterns for resource allocation.

**Tasks:**
1. Load the daily NYC accident data (provided below)
2. Create an autocorrelation plot and identify the key periodicities
3. Decompose the time series using `seasonal_decompose` with an appropriate period
4. What days of the week have the highest/lowest accident rates?

**Hint:** For daily data with weekly seasonality, use `period=7`

In [ ]:
# Load daily accident data
sql = '''
  SELECT DATE_TRUNC(DATE_TIME, DAY) AS acc_date, COUNT(*) AS accidents
  FROM collisions.collisions
  GROUP BY acc_date
  ORDER BY acc_date
'''

acc = client.query(sql).to_dataframe()
acc['acc_date'] = pd.to_datetime(acc['acc_date'])
acc = acc.set_index('acc_date')

acc.plot(title='NYC Daily Accidents')

In [ ]:
# Your analysis here


### Activity 2 Solution

In [ ]:
#@title Activity 2 Solution (click to expand)

# Step 1: Autocorrelation analysis
print("Key autocorrelations:")
print(f"  Lag 1 (day-to-day): {acc['accidents'].autocorr(lag=1):.3f}")
print(f"  Lag 7 (week-to-week): {acc['accidents'].autocorr(lag=7):.3f}")
print(f"  Lag 365 (year-to-year): {acc['accidents'].autocorr(lag=365):.3f}")

# ACF plot
plot = pd.plotting.autocorrelation_plot(acc['accidents'])
plot.set_xlim(0, 90)
plt.title('Autocorrelation (first 90 days)')
plt.show()

# Step 2: Seasonal decomposition with period=7 (weekly)
decomposition = seasonal_decompose(
    x=acc['accidents'],
    model='multiplicative',
    period=7,
    extrapolate_trend=7
)

fig = plt.figure(figsize=(10, 8))
fig = decomposition.plot(
    observed=False,
    seasonal=True,
    trend=True,
    resid=True,
)
plt.show()

# Step 3: Weekly pattern analysis
print("\nWeekly Seasonal Pattern:")
weekly_pattern = decomposition.seasonal.head(7)
weekly_pattern.index = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
print(weekly_pattern)

weekly_pattern.plot(kind='bar', title='Weekly Seasonality Pattern')
plt.ylabel('Seasonal Factor (1.0 = average)')
plt.show()

print("\nKey Finding: Accidents are highest on Fridays and lowest on Sundays.")
print("This suggests more traffic enforcement resources should be deployed on Fridays.")

---
## Activity 3: ETF Sector Analysis

**Scenario:** You're a portfolio analyst comparing the performance of different market sectors.

Here are sector ETFs representing different areas of the economy (GICS classification):

| Ticker | Sector | Description |
|--------|--------|-------------|
| XLK | Technology | Tech companies |
| XLV | Healthcare | Pharma, medical devices |
| XLF | Financials | Banks, insurance |
| XLE | Energy | Oil, gas, renewables |
| XLRE | Real Estate | REITs |

**Tasks:**
1. Download these 5 ETFs since January 1, 2010
2. Normalize their prices to compare relative performance
3. Calculate daily returns and examine their correlations
4. Which sectors move together? Which provide diversification?
5. Visualize the return correlation using pairwise scatterplots (use the a `seaborn.pairplot(... kind='reg')` command).



In [ ]:
etf_tickers = ['XLK', 'XLV', 'XLF', 'XLE', 'XLRE']
etf_names = ['Tech', 'Healthcare', 'Financials', 'Energy', 'Real Estate']

# Your code here

# Here is a command that makes the regression line red, and makes
# the scatterplots more transparent.

# sns.pairplot(returns, kind='reg', plot_kws={'line_kws':{'color':'red'}, 'scatter_kws': {'alpha': 0.01}}`


### Activity 3 Solution

In [ ]:
#@title Activity 3 Solution (click to expand)

# Step 1: Download ETF data
etf_tickers = ['XLK', 'XLV', 'XLF', 'XLE', 'XLRE']
etf_names = {
    'XLK':'Tech',
    'XLV':'Healthcare',
    'XLF':'Financials',
    'XLE':'Energy',
    'XLRE':'Real Estate'
}

etf_df = yf.download(
    tickers=etf_tickers,
    start='2010-01-01',
    auto_adjust=True,
    progress=False
)['Close']

# Put user-friendly column names
etf_df = etf_df.rename(columns=etf_names)

# Step 2: Normalize prices
first_values = etf_df.bfill().iloc[0] # backfill NaN values and pick first row
normalized_etf = etf_df / first_values

normalized_etf.plot(figsize=(10, 4), title='Normalized ETF Performance (Base = Jan 2010)')
plt.ylabel('Growth Multiple')
plt.show()

# Step 3: Calculate returns and correlations
returns_etf = etf_df.pct_change().dropna()
returns_etf.columns = etf_names

print("Return Statistics:")
display(returns_etf.describe().round(4))

# Step 4: Correlation matrix
print("\nCorrelation Matrix:")
corr_matrix = returns_etf.corr()
display(corr_matrix.round(3))

# Visualize with heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='RdYlBu_r', center=0, fmt='.2f')
plt.title('Sector Return Correlations')
plt.show()

print("\nKey Insights:")
print("- Energy has the lowest correlation with other sectors → good for diversification")
print("- Tech, Healthcare, and Financials are moderately correlated")
print("- Real Estate correlates most with Financials (both interest-rate sensitive)")

---
## Part 8: From Analysis to Forecasting (Preview)

Everything we've learned so far—identifying trends, seasonality, and autocorrelation—sets the foundation for **forecasting**. While detailed forecasting is beyond this notebook, here's a preview of how these concepts connect.

### Simple Forecasting Approaches

| Method | Idea | When to Use |
|--------|------|-------------|
| **Naive** | Next value = last value | Random walk data (like stock prices) |
| **Seasonal Naive** | Next value = same period last year | Strong seasonality, stable trend |
| **Trend + Seasonal** | Extrapolate trend, add seasonal pattern | Clear trend and seasonality |

In [ ]:
# Simple seasonal naive forecast for wine sales
# Forecast: next month's sales = same month last year's sales

decomposition = seasonal_decompose(
    x = df['Sales'],
    model='multiplicative',
    period=12,
    extrapolate_trend=24
)

# Get the last 12 months of actual data
last_year = df['Sales'].tail(12)

# Use decomposition to make a simple forecast
# Forecast = Latest Trend × Seasonal Factor

latest_trend = decomposition.trend.dropna().iloc[-1]
seasonal_factors = decomposition.seasonal.tail(12)

# Create forecast for next 12 months
forecast = latest_trend * seasonal_factors.values

# Plot
plt.figure(figsize=(10, 4))
df['Sales'].tail(36).plot(label='Actual', marker='o')
pd.Series(
    forecast,
    index=pd.date_range(start=df.index[-1] + pd.DateOffset(months=1), periods=12, freq='MS')
).plot(label='Simple Forecast', linestyle='--', marker='s')
plt.title('Simple Seasonal Forecast')
plt.legend()
plt.show()

print("This simple forecast assumes the trend stays flat.")
print("More sophisticated methods (ARIMA, Prophet, etc.) can model trend changes.")

### Next Steps for Forecasting

To build production-quality forecasts, you would typically:

1. **Split data** into training and test sets (hold out recent data)
2. **Choose a model** based on the patterns you've identified:
   - ARIMA for autocorrelated data
   - Prophet for data with multiple seasonalities
   - Exponential smoothing for trending data
3. **Evaluate forecast accuracy** using metrics like MAPE (Mean Absolute Percentage Error)
4. **Generate prediction intervals** to quantify uncertainty

The decomposition and autocorrelation analysis we've done here tells you *which* model to choose and *what* parameters to use.

---
## Summary: Key Concepts and Commands

### Time Series Data Handling

| Task | Command |
|------|--------|
| Convert to datetime | `pd.to_datetime(df['column'])` |
| Set datetime index | `df.set_index('date_column')` |
| Resample to different frequency | `df.resample('W').sum()` |
| Percentage change | `df.pct_change()` |

### Resampling Frequency Codes

| Code | Meaning |
|------|--------|
| `'D'` | Daily |
| `'W'` | Weekly |
| `'MS'` / `'ME'` | Month start / end |
| `'QE'` | Quarter end |
| `'YE'` | Year end |

### Analysis Tools

| Task | Command |
|------|--------|
| Autocorrelation at specific lag | `series.autocorr(lag=n)` |
| Lag plot | `pd.plotting.lag_plot(series, lag=n)` |
| Autocorrelation plot | `pd.plotting.autocorrelation_plot(series)` |
| Seasonal decomposition | `seasonal_decompose(series, model='multiplicative', period=n)` |

### Window Functions

| Task | Command |
|------|--------|
| Rolling window | `series.rolling(n).mean()` |
| Expanding window | `series.expanding().mean()` |
| Exponential weighting | `series.ewm(halflife=n).mean()` |

---
## Appendix (Optional): Periodogram Analysis

The **periodogram** is a tool from signal processing that identifies the most important periodicities in data. It's typically applied to detrended data. For most business data, daily, weekly, and yearly are the three periods to consider—but a periodogram can reveal unexpected patterns.

In [ ]:
from scipy.signal import periodogram

# Create a periodogram from the detrended sales time series
timeseries = df["Sales"] / decomposition.trend
freqs, psd = periodogram(timeseries.dropna())

# Create a dataframe with the results
# Convert frequencies to periods (time between events)
prd = pd.DataFrame({"freqs": freqs, "psd": psd})
prd['period'] = 1/prd['freqs']

# Plot the results
prd.plot(
    x='period',
    y='psd',
    ylabel='Power Spectral Density',
    xlabel='Period (months)',
    xlim=(0, 24),
    # logx=True,
    title='Periodogram of Wine Sales'
)

print("The peak at period ≈ 12 confirms the strong annual seasonality.")